In [117]:
import mysql.connector  as connection
import pandas as pd
import os


In [118]:
# Define files location
db_credentials_dir = 'C:/Users/User/Desktop/Moje dokumenty/Raporty obce/Kacper/63_issue_158946_payments/'
sql_dir = 'C:/Users/User/AppData/Roaming/DBeaverData/workspace6/General/Scripts/'
file_dir = 'C:/Users/User/Desktop/Moje dokumenty/Raporty obce/Kacper/63_issue_158946_payments/Excel/2022/'

In [119]:
# Run MySQL query to fetch auftrags. To refresh data, keep in mind to set correct dates in MySQL query
# Putty must be running while running the script
with open(db_credentials_dir + 'db_credentials.txt', 'r') as f:
    db_credentials = f.read().splitlines()

try:
    db_connections = connection.connect(
        host=db_credentials[0],
        user=db_credentials[1],
        password=db_credentials[2],
        database=db_credentials[3]
    )
    sql_file = open(sql_dir + 'Script-19.sql')
    sql_query = sql_file.read()
    auctions = pd.read_sql(sql_query,db_connections)

    # close the connection
    db_connections.close()  
except Exception as e:
    db_connections.close()
    print(str(e))

C:\Users\User\AppData\Local\Temp\ipykernel_4272\3265242886.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  auctions = pd.read_sql(sql_query,db_connections)


In [120]:
# Open accounting file
in_books_22 = pd.read_csv(file_dir + 'in_books_22.csv')

In [121]:
# Clean auctions data
auctions['auction_number'] = auctions['auction_number'].astype(str)
auctions['auction_number'] = auctions['auction_number'].astype(str).replace(r'\.0', '', regex=True)
auctions['invoice_number'] = auctions['invoice_number'].astype(str)
auctions['invoice_number'] = auctions['invoice_number'].astype(str).replace(r'\.0', '', regex=True)

In [122]:
# Filter all rows that need auction numbers to be found
in_books_22_filt = in_books_22[in_books_22['auction_number'].isna()]

In [123]:
# Extract invoice number
def extract_number(text):
    text_parts = text.split(' ')
    number = text_parts[1] if len(text_parts) > 1 else None
    return number

in_books_22_filt['invoice_number'] = in_books_22_filt['Text'].apply(extract_number)


C:\Users\User\AppData\Local\Temp\ipykernel_4272\3609727072.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_books_22_filt['invoice_number'] = in_books_22_filt['Text'].apply(extract_number)


In [124]:
# Drop auction_number column with NaN values 
in_books_22_filt = in_books_22_filt.drop('auction_number', axis=1)

In [125]:
# Add column with auction number 
merged = pd.merge(in_books_22_filt, auctions, on='invoice_number', how='left')

In [126]:
# Drop all rows with NaN values from original file from accounting
in_books_22_no_na = in_books_22.dropna(subset=["auction_number"]) 

In [127]:
# Union both dataframes into one
union_books = pd.concat([in_books_22_no_na, merged])

In [128]:
# Run MySQL query to fetch tickets. To refresh data, keep in mind to set correct dates in MySQL query
# Putty must be running while running the script
with open(db_credentials_dir + 'db_credentials.txt', 'r') as f:
    db_credentials = f.read().splitlines()

try:
    db_connections = connection.connect(
        host=db_credentials[0],
        user=db_credentials[1],
        password=db_credentials[2],
        database=db_credentials[3]
    )
    tickets_sql = open(file_dir + 'tickets_sql.sql')
    tickets_sql_query = tickets_sql.read()
    all_tickets_sql = pd.read_sql(tickets_sql_query,db_connections)

    # close the connection
    db_connections.close()  
except Exception as e:
    db_connections.close()
    print(str(e))

C:\Users\User\AppData\Local\Temp\ipykernel_4272\2179302637.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_tickets_sql = pd.read_sql(tickets_sql_query,db_connections)


In [129]:
# Filter all rows that need auction numbers to be found
in_books_22_filt = union_books[union_books['auction_number'].isna()]

In [130]:
# to filter out ticket numbers with space
in_books_22_split = in_books_22_filt
in_books_22_split['ticket_number'] = in_books_22_filt['Text'].str.extract(r'(\d+)')
in_books_22_split = in_books_22_split.dropna(subset=["ticket_number"]) 

C:\Users\User\AppData\Local\Temp\ipykernel_4272\1225880865.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_books_22_split['ticket_number'] = in_books_22_filt['Text'].str.extract(r'(\d+)')


In [131]:
# to filter out ticket numbers without space
in_books_22_comb = in_books_22_filt
in_books_22_comb['ticket_number'] = in_books_22_filt['Text'].str.extract(r'ticket(\d+)')
in_books_22_comb = in_books_22_comb.dropna(subset=["ticket_number"]) 

C:\Users\User\AppData\Local\Temp\ipykernel_4272\248470219.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_books_22_comb['ticket_number'] = in_books_22_filt['Text'].str.extract(r'ticket(\d+)')


In [132]:
# all tickets numbers that need auction number to be added
union_ticket = pd.concat([in_books_22_split, in_books_22_comb])

In [133]:
# cleare those where ticket number were not present
union_ticket = union_ticket.drop(['auction_number', 'txnid'], axis=1)

In [134]:
# prepare df for join
union_ticket['ticket_number'] = union_ticket['ticket_number'].astype(str)
union_ticket['ticket_number'] = union_ticket['ticket_number'].str.strip()
all_tickets_sql['ticket_number'] = all_tickets_sql['ticket_number'].astype(str)
all_tickets_sql['ticket_number'] = all_tickets_sql['ticket_number'].str.strip()

In [135]:
# join auction number to ticket number
tickets = pd.merge(union_ticket, all_tickets_sql, on='ticket_number', how='left')

In [136]:
# prepare df for cleaning
tickets['Text'] = tickets['Text'].astype(str)
tickets['Text'] = tickets['Text'].str.strip()
in_books_22_filt['Text'] = in_books_22_filt['Text'].astype(str)
in_books_22_filt['Text'] = in_books_22_filt['Text'].str.strip()

C:\Users\User\AppData\Local\Temp\ipykernel_4272\2423840857.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_books_22_filt['Text'] = in_books_22_filt['Text'].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_4272\2423840857.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_books_22_filt['Text'] = in_books_22_filt['Text'].str.strip()


In [137]:
# find all values where I was unable to match any auction number that were truncated
not_in_tickets = in_books_22_filt[~in_books_22_filt['Text'].isin(tickets['Text'])]

In [138]:
# union dataframes
tickets_all = pd.concat([not_in_tickets, tickets])

In [143]:
# remove values that were taken for processing
union_books2 = union_books.dropna(subset=["auction_number"]) 

In [162]:
# add values that were processed 
union_books = pd.concat([tickets_all, union_books2])

In [151]:
# Read data from Power BI
FACT_all_transactions = pd.read_csv(file_dir + 'FACT_all_transactions.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_4272\691017383.py:2: DtypeWarning: Columns (2,14) have mixed types. Specify dtype option on import or set low_memory=False.
  FACT_all_transactions = pd.read_csv(file_dir + 'FACT_all_transactions.csv')


In [152]:
# Split auction number
FACT_all_transactions[['auction_number', 'txnid']] = FACT_all_transactions['Main auction'].str.split('_', expand=True)

In [153]:
# Change data type
FACT_all_transactions['Transaction_Date'] = pd.to_datetime(FACT_all_transactions['Transaction_Date'])

In [154]:
# Filter 2022
FACT_all_transactions_2022= FACT_all_transactions[(FACT_all_transactions['Transaction_Date'] >= pd.Timestamp('2022-01-01')) & (FACT_all_transactions['Transaction_Date'] < pd.Timestamp('2023-01-01')) ]

In [179]:
# prepare df for cleaning
union_books['auction_number'] = union_books['auction_number'].astype(str)
FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].astype(str)
union_books['auction_number'] = union_books['auction_number'].astype(str).replace(r'\.0', '', regex=True)
FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].astype(str).replace(r'\.0', '', regex=True)
FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].str.strip()
union_books['auction_number'] = union_books['auction_number'].str.strip()

C:\Users\User\AppData\Local\Temp\ipykernel_4272\3082652090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].astype(str)
C:\Users\User\AppData\Local\Temp\ipykernel_4272\3082652090.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FACT_all_transactions_2022['auction_number'] = FACT_all_transactions_2022['auction_number'].astype(str).replace(r'\.0', '', regex=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4272\30

In [180]:
# substract all rows that are present in Power Bi
in_books_22_fixed = union_books[~union_books['auction_number'].isin(FACT_all_transactions_2022['auction_number'])]

In [184]:
# Save into the csv file
in_books_22_fixed.to_csv(file_dir + 'in_books_22_fixed.csv', index=True, encoding='utf-8')